# Codificação de variáveis categóricas

Neste caderno, apresentaremos maneiras típicas de lidar com
**variáveis categóricas** codificando-as, ou seja, **codificação ordinal** e
**codificação one-hot**.

Vamos primeiro carregar todo o conjunto de dados adulto contendo números e
Dados categóricos. já vamos agora retirar o alvo dos dados que usaremos para treinar nosso modelo preditivo.

In [1]:
import pandas as pd
 
adult_census = pd.read_csv("adult-census.csv")
# drop the duplicated column `"education-num"` as stated in the first notebook
adult_census = adult_census.drop(columns=['ID','fnlwgt:','education-num:'])

target_name = "class"
target = adult_census[target_name]

data = adult_census.drop(columns=target_name)

adult_census.head()

,age,workclass,education:,marital-status:,occupation:,relationship:,race:,sex:,capital-gain:,capital-loss:,hours-per-week:,native-country:,class
0,39,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


## Identificar variáveis categóricas

Como vimos na seção anterior, uma variável numérica é um
quantidade representada por um número real ou inteiro. Essas variáveis podem ser
naturalmente tratada por algoritmos de aprendizado de máquina que são normalmente compostos
de uma sequência de instruções aritméticas, como adições e
multiplicações.

Em contraste, as variáveis categóricas têm valores discretos, normalmente
representado por rótulos de string (mas não apenas) retirados de uma lista finita de
escolhas possíveis. Por exemplo, a variável `native-country` em nosso conjunto de dados
é uma variável categórica porque codifica os dados usando uma lista finita de
países possíveis (junto com o símbolo `?` quando esta informação é
ausência de):

In [2]:
data["native-country:"].value_counts().sort_index()

?                               583
Cambodia                         19
Canada                          121
China                            75
Columbia                         59
Cuba                             95
Dominican-Republic               70
Ecuador                          28
El-Salvador                     106
England                          90
France                           29
Germany                         137
Greece                           29
Guatemala                        64
Haiti                            44
Holand-Netherlands                1
Honduras                         13
Hong                             20
Hungary                          13
India                           100
Iran                             43
Ireland                          24
Italy                            73
Jamaica                          81
Japan                            62
Laos                             18
Mexico                          643
Nicaragua                   

Como podemos reconhecer facilmente colunas categóricas entre o conjunto de dados? Parte de
a resposta está no tipo de dados das colunas:

In [3]:
data.dtypes

age                 int64
workclass          object
education:         object
marital-status:    object
occupation:        object
relationship:      object
race:              object
sex:               object
capital-gain:       int64
capital-loss:       int64
hours-per-week:     int64
native-country:    object
dtype: object

Se olharmos para a coluna `" native-country: "`, observamos que seu tipo de dados é
`objeto`, o que significa que contém valores de string.

## Selecione recursos com base em seus tipos de dados

No bloco de notas anterior, definimos manualmente as colunas numéricas. Poderíamos
faça uma abordagem semelhante. Em vez disso, usaremos a função auxiliar scikit-learn
`make_column_selector`, que nos permite selecionar colunas com base em
seu tipo de dados. Vamos ilustrar como usar esse auxiliar.

In [4]:
from sklearn.compose import make_column_selector as selector

categorical_columns_selector = selector(dtype_include=object)
categorical_columns = categorical_columns_selector(data)
categorical_columns

['workclass',
 'education:',
 'marital-status:',
 'occupation:',
 'relationship:',
 'race:',
 'sex:',
 'native-country:']

Aqui, criamos o seletor passando o tipo de dados a incluir; nós então
passou o conjunto de dados de entrada para o objeto seletor, que retornou uma lista de
nomes de colunas que possuem o tipo de dados solicitado. Agora podemos filtrar o
colunas indesejadas:

In [5]:
data_categorical = data[categorical_columns]
data_categorical.head()

,workclass,education:,marital-status:,occupation:,relationship:,race:,sex:,native-country:
0,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,United-States
1,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States
2,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,United-States
3,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,United-States
4,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,Cuba


In [6]:
print(f"O dataset é composto de {data_categorical.shape[1]} features")

O dataset é composto de 8 features


No restante desta seção, apresentaremos diferentes estratégias para
codificar dados categóricos em dados numéricos que podem ser usados por um
algoritmo de aprendizado de máquina.

## Strategies to encode categories

### Encoding ordinal categories

A estratégia mais intuitiva é codificar cada categoria com um diferente
número. O `OrdinalEncoder` irá transformar os dados dessa maneira.
Começaremos codificando uma única coluna para entender como a codificação
trabalho.

In [7]:
from sklearn.preprocessing import OrdinalEncoder

education_column = data_categorical[["education:"]]

encoder = OrdinalEncoder()
education_encoded = encoder.fit_transform(education_column)
education_encoded

array([[ 9.],
       [ 9.],
       [11.],
       ...,
       [11.],
       [11.],
       [11.]])

Vemos que cada categoria em `" educação "` foi substituída por um numérico
valor. Poderíamos verificar o mapeamento entre as categorias e os números
valores verificando o atributo ajustado `categorias_`.

In [8]:
encoder.categories_

[array(['10th', '11th', '12th', '1st-4th', '5th-6th', '7th-8th', '9th',
        'Assoc-acdm', 'Assoc-voc', 'Bachelors', 'Doctorate', 'HS-grad',
        'Masters', 'Preschool', 'Prof-school', 'Some-college'],
       dtype=object)]

Agora, podemos verificar a codificação aplicada em todos os recursos categóricos.

In [9]:
data_encoded = encoder.fit_transform(data_categorical)
data_encoded[:5]

array([[ 7.,  9.,  4.,  1.,  1.,  4.,  1., 39.],
       [ 6.,  9.,  2.,  4.,  0.,  4.,  1., 39.],
       [ 4., 11.,  0.,  6.,  1.,  4.,  1., 39.],
       [ 4.,  1.,  2.,  6.,  0.,  2.,  1., 39.],
       [ 4.,  9.,  2., 10.,  5.,  2.,  0.,  5.]])

In [10]:
encoder.categories_

[array(['?', 'Federal-gov', 'Local-gov', 'Never-worked', 'Private',
        'Self-emp-inc', 'Self-emp-not-inc', 'State-gov', 'Without-pay'],
       dtype=object),
 array(['10th', '11th', '12th', '1st-4th', '5th-6th', '7th-8th', '9th',
        'Assoc-acdm', 'Assoc-voc', 'Bachelors', 'Doctorate', 'HS-grad',
        'Masters', 'Preschool', 'Prof-school', 'Some-college'],
       dtype=object),
 array(['Divorced', 'Married-AF-spouse', 'Married-civ-spouse',
        'Married-spouse-absent', 'Never-married', 'Separated', 'Widowed'],
       dtype=object),
 array(['?', 'Adm-clerical', 'Armed-Forces', 'Craft-repair',
        'Exec-managerial', 'Farming-fishing', 'Handlers-cleaners',
        'Machine-op-inspct', 'Other-service', 'Priv-house-serv',
        'Prof-specialty', 'Protective-serv', 'Sales', 'Tech-support',
        'Transport-moving'], dtype=object),
 array(['Husband', 'Not-in-family', 'Other-relative', 'Own-child',
        'Unmarried', 'Wife'], dtype=object),
 array(['Amer-Indian-Eskimo'

In [11]:
print(f"O dataset encoded contém {data_encoded.shape[1]} features")

O dataset encoded contém 8 features


Vemos que as categorias foram codificadas para cada recurso (coluna)
independentemente. Também observamos que o número de recursos antes e depois do
a codificação é a mesma.

No entanto, tome cuidado ao aplicar esta estratégia de codificação:
usar esta representação inteira leva a modelos preditivos downstream
assumir que os valores estão ordenados (0 <1 <2 <3 ... por exemplo).

Por padrão, `OrdinalEncoder` usa uma estratégia lexicográfica para mapear string
rótulos de categoria para inteiros. Esta estratégia é arbitrária e frequentemente
sem significado. Por exemplo, suponha que o conjunto de dados tenha uma variável categórica
denominado `" tamanho "` com categorias como "S", "M", "L", "XL". Nós gostaríamos de
representação inteira para respeitar o significado dos tamanhos, mapeando-os para
números inteiros crescentes, como `0, 1, 2, 3`.
No entanto, a estratégia lexicográfica usada por padrão mapearia os rótulos
"S", "M", "L", "XL" a 2, 1, 0, 3, seguindo a ordem alfabética.

A classe `OrdinalEncoder` aceita um argumento de construtor de` categorias` para
passe categorias na ordem esperada explicitamente. Você pode encontrar mais
informação no [scikit-learn documentation](https://scikit-learn.org/stable/modules/preprocessing.html#encoding-categorical-features) if needed.

### Encoding nominal categories (without assuming any order)

`OneHotEncoder` é um codificador alternativo que impede o downstream
modelos para fazer uma suposição falsa sobre a ordem das categorias. Para
determinado recurso, ele criará tantas novas colunas quanto possível
categorias. Para uma determinada amostra, o valor da coluna correspondente ao
categoria será definida como `1` enquanto todas as colunas das outras categorias
será definido como `0`.

Começaremos codificando um único recurso (por exemplo, `" educação "`) para ilustrar
como funciona a codificação.

In [12]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse=False)
education_encoded = encoder.fit_transform(education_column)
education_encoded

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

<div class="admonition note alert alert-info">
<p class="first admonition-title" style="font-weight: bold;">Note</p>
<p><tt class="docutils literal">sparse=False</tt> é usado no <tt class="docutils literal">OneHotEncoder</tt>para fins didáticos, a saber visualização mais fácil dos dados.</p>
<p class="last">Matrizes esparsas são estruturas de dados eficientes quando a maior parte de sua matriz
os elementos são zero. Eles não serão abordados em detalhes neste curso. Se você
quer mais detalhes sobre eles, você pode olhar
<a class="reference external" href="https://scipy-lectures.org/advanced/scipy_sparse/introduction.html#why-sparse-matrices">esta</a>.</p>
</div>

Vemos que a codificação de um único recurso dará um array NumPy cheio de zeros
e uns. Podemos obter um melhor entendimento usando o recurso associado
nomes resultantes da transformação.

In [13]:
feature_names = encoder.get_feature_names(input_features=["education"])
education_encoded = pd.DataFrame(education_encoded, columns=feature_names)
education_encoded

,education_10th,education_11th,education_12th,education_1st-4th,education_5th-6th,education_7th-8th,education_9th,education_Assoc-acdm,education_Assoc-voc,education_Bachelors,education_Doctorate,education_HS-grad,education_Masters,education_Preschool,education_Prof-school,education_Some-college
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32557,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
32558,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
32559,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


Como podemos ver, cada categoria (valor único) tornou-se uma coluna; a codificação
retornou, para cada amostra, um 1 para especificar a qual categoria ela pertence.

Vamos aplicar essa codificação no conjunto de dados completo.

In [14]:
print(f"O dataset é composto de {data_categorical.shape[1]} features")
data_categorical.head()

O dataset é composto de 8 features


,workclass,education:,marital-status:,occupation:,relationship:,race:,sex:,native-country:
0,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,United-States
1,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States
2,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,United-States
3,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,United-States
4,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,Cuba


In [15]:
data_encoded = encoder.fit_transform(data_categorical)
data_encoded[:5]

array([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

In [16]:
print(f"O encoded dataset contém {data_encoded.shape[1]} features")

O encoded dataset contém 102 features


Vamos envolver este array NumPy em um dataframe com nomes de coluna informativos como
fornecido pelo objeto codificador:

In [17]:
columns_encoded = encoder.get_feature_names(data_categorical.columns)
pd.DataFrame(data_encoded, columns=columns_encoded).head()

,workclass_?,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay,education:_10th,...,native-country:_Portugal,native-country:_Puerto-Rico,native-country:_Scotland,native-country:_South,native-country:_Taiwan,native-country:_Thailand,native-country:_Trinadad&Tobago,native-country:_United-States,native-country:_Vietnam,native-country:_Yugoslavia
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Veja como a variável `" workclass "` dos 3 primeiros registros tem sido
codificado e compare-o com a representação da string original.

O número de recursos após a codificação é mais de 10 vezes maior do que
nos dados originais porque algumas variáveis como `ocupação` e
`native-country` tem muitas categorias possíveis.

### Escolhendo uma estratégia de encoding (codificação)

A escolha de uma estratégia de codificação dependerá dos modelos subjacentes e do
tipo de categorias (ou seja, ordinal vs. nominal).

Na verdade, usar um `OrdinaleEncoder` produzirá categorias ordinais. Isso significa
que há uma ordem nas categorias resultantes (por exemplo, `0> 1> 2`). O
impacto de violar esta suposição de pedido é realmente dependente do
modelos a jusante. Modelos lineares serão afetados por categorias desordenadas
enquanto os modelos baseados em árvore não serão.

Assim, em geral, `OneHotEncoder` é a estratégia de codificação usada quando o
modelos downstream são ** modelos lineares ** enquanto `OrdinalEncoder` é usado com
** modelos baseados em árvore **.

Você ainda pode usar um `OrdinalEncoder` com modelos lineares, mas precisa ser
claro que:
- as categorias originais (antes da codificação) têm uma ordem;
- as categorias codificadas seguem a mesma ordem do original
  categorias.
O próximo exercício destaca a questão do uso indevido de `OrdinalEncoder` com um
modelo linear.

Além disso, não há necessidade de usar um `OneHotEncoder`, mesmo se o original
as categorias não têm uma ordem dada com o modelo baseado em árvore. Será
o objetivo do exercício final desta seqüência.

## Avalie nosso pipeline de previsão

Agora podemos integrar este codificador dentro de um pipeline de aprendizado de máquina como fazemos
fez com dados numéricos: vamos treinar um classificador linear nos dados codificados
e verificar o desempenho estatístico deste pipeline de aprendizado de máquina usando
validação cruzada.

Antes de criar o pipeline, temos que nos demorar no `native-country`.
Vamos relembrar algumas estatísticas sobre esta coluna.

In [18]:
data["native-country:"].value_counts()

United-States                 29170
Mexico                          643
?                               583
Philippines                     198
Germany                         137
Canada                          121
Puerto-Rico                     114
El-Salvador                     106
India                           100
Cuba                             95
England                          90
Jamaica                          81
South                            80
China                            75
Italy                            73
Dominican-Republic               70
Vietnam                          67
Guatemala                        64
Japan                            62
Poland                           60
Columbia                         59
Taiwan                           51
Haiti                            44
Iran                             43
Portugal                         37
Nicaragua                        34
Peru                             31
Greece                      

Vemos que a categoria `Holand-Netherlands` ocorre raramente. Isso vai
ser um problema durante a validação cruzada: se a amostra acabar no conjunto de teste
durante a divisão, o classificador não teria visto a categoria durante
treinamento e não será capaz de codificá-lo.

No scikit-learn, existem duas soluções para contornar esse problema:

* liste todas as categorias possíveis e forneça-as ao codificador por meio do
  argumento de palavra-chave `categories`;
* use o parâmetro `handle_unknown`.

Aqui, usaremos a última solução para simplificar.

<div class="admonition tip alert alert-warning">
<p class="first admonition-title" style="font-weight: bold;">Tip</p>
<p class="last">Esteja ciente do<tt class="docutils literal">OrdinalEncoder</tt> expõe também um parâmetro
<tt class="docutils literal">handle_unknown</tt>. Pode ser definido para <tt class="docutils literal">use_encoded_value</tt> e definindo
<tt class="docutils literal">unknown_value</tt> para lidar com categorias raras. Você vai usar estes
parâmetros no próximo exercício.</p>
</div>

Agora podemos criar nosso pipeline de machine learning pipeline.

In [19]:
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression

model = make_pipeline(OneHotEncoder(handle_unknown="ignore"), LogisticRegression(max_iter=500))

<div class="admonition note alert alert-info">
<p class="first admonition-title" style="font-weight: bold;">Note</p>
<p class="last">Aqui, precisamos aumentar o número máximo de iterações para obter uma
convergiu <tt class="docutils literal">LogisticRegression</tt>e silêncio a 
<tt class="docutils literal">ConvergenceWarning</tt>.Contrário
para os recursos numéricos, os recursos categóricos codificados em um único ponto são todos
na mesma escala (os valores são 0 ou 1), então eles não se beneficiariam de
dimensionamento. Neste caso, aumentando <tt class="docutils literal">max_iter</tt>é a coisa certa a fazer.</p>
</div>

Finalmente, podemos verificar o desempenho estatístico do modelo apenas usando o
colunas categóricas.

In [20]:
from sklearn.model_selection import cross_validate
cv_results = cross_validate(model, data_categorical, target)
cv_results

{'fit_time': array([0.54907203, 0.81150579, 0.8360033 , 0.86592054, 1.16543698]),
 'score_time': array([0.02040768, 0.02506971, 0.02497864, 0.02527404, 0.04625821]),
 'test_score': array([0.82634731, 0.82939189, 0.83077396, 0.83937346, 0.83215602])}

In [21]:
scores = cv_results["test_score"]
print(f"A accuracy é: {scores.mean():.3f} +/- {scores.std():.3f}")

A accuracy é: 0.832 +/- 0.004


Como você pode ver, esta representação das variáveis categóricas é
ligeiramente mais preditivo da receita do que as variáveis numéricas
que usamos anteriormente.

Neste caderno temos:

* viu duas estratégias comuns para codificar recursos categóricos: **ordinal
  codificação** e **codificação one-hot**;
* usou um **pipeline** para usar um **codificador one-hot** antes de ajustar uma logística
  regressão.

# 📝 Exercício M1.04

O objetivo deste exercício é avaliar o impacto do uso de um
codificação inteira para variáveis categóricas junto com uma
modelo de classificação como Regressão Logística.

Para fazer isso, vamos tentar usar `OrdinalEncoder` para pré-processar o categórico
variáveis. Este pré-processador é montado em um pipeline com
`LogisticRegression`. O desempenho estatístico do pipeline pode ser
avaliada por validação cruzada e depois comparada com a pontuação obtida quando
usando `OneHotEncoder` ou alguma outra pontuação de linha de base.

Primeiro, carregamos o conjunto de dados.

In [22]:
import pandas as pd

adult_census = pd.read_csv("adult-census.csv")
# drop the duplicated column `"education-num"` as stated in the first notebook
adult_census = adult_census.drop(columns=['ID','fnlwgt:','education-num:'])

target_name = "class"
target = adult_census[target_name]

data = adult_census.drop(columns=target_name)

adult_census.head()

,age,workclass,education:,marital-status:,occupation:,relationship:,race:,sex:,capital-gain:,capital-loss:,hours-per-week:,native-country:,class
0,39,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


No bloco de notas anterior, usamos `sklearn.compose.make_column_selector` para
seleciona automaticamente colunas com um tipo de dados específico (também chamado de `dtype`).
Aqui, usaremos este seletor para obter apenas as colunas contendo strings
(coluna com dtipo `objeto`) que correspondem a características categóricas em nosso
conjunto de dados.

In [23]:
from sklearn.compose import make_column_selector as selector

categorical_columns_selector = selector(dtype_include=object)
categorical_columns = categorical_columns_selector(data)
data_categorical = data[categorical_columns]

Filtramos nosso conjunto de dados para que ele contenha apenas características categóricas.
Defina um pipeline scikit-learn composto por um `OrdinalEncoder` e um
Classificador `LogisticRegression`.

Porque `OrdinalEncoder` pode gerar erros se encontrar uma categoria desconhecida em
tempo de previsão, você pode definir o `handle_unknown =" use_encoded_value "` e
Parâmetros `unknown_value`. Você pode consultar o
[scikit-learn documentation](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html)
para obter mais detalhes sobre esses parâmetros.

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OrdinalEncoder
from sklearn.linear_model import LogisticRegression

# Write your code here.
model = make_pipeline(
    OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1),
    LogisticRegression(max_iter=500))

Seu modelo agora está definido. Avalie-o usando uma validação cruzada usando
`sklearn.model_selection.cross_validate`.

In [25]:
from sklearn.model_selection import cross_validate

# Write your code here.
cv_results = cross_validate(model, data_categorical, target)

scores = cv_results["test_score"]
print("The mean cross-validation accuracy is: "
      f"{scores.mean():.3f} +/- {scores.std():.3f}")

The mean cross-validation accuracy is: 0.832 +/- 0.004


Agora, gostaríamos de comparar o desempenho estatístico de nossos anteriores
modelo com um novo modelo onde, em vez de usar um `OrdinalEncoder`, iremos
use um `OneHotEncoder`. Repita a avaliação do modelo usando validação cruzada.
Compare a pontuação de ambos os modelos e conclua sobre o impacto da escolha de um
estratégia de codificação específica ao usar um modelo linear.

In [26]:
from sklearn.preprocessing import OneHotEncoder

# Write your code here.
model = make_pipeline(OneHotEncoder(handle_unknown="ignore"),
                      LogisticRegression(max_iter=500))
cv_results = cross_validate(model, data_categorical, target)
scores = cv_results["test_score"]
print("The mean cross-validation accuracy is: "
      f"{scores.mean():.3f} +/- {scores.std():.3f}")

The mean cross-validation accuracy is: 0.832 +/- 0.004
